<a href="https://colab.research.google.com/github/codingsamurai-10/DataScience/blob/master/Amazon_Price_Drop_Notifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Price drop notifier
The aim of this project is to automatically scrape data from a specified page on Amazon and notify the user on Slack in case there is a price drop.

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from json import dumps
from httplib2 import Http

In [2]:
sleep_time = 10
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
slack_url = 'YOUR_SLACK_WEBHOOK_URL'
url_to_scrape = 'PRODUCT_PAGE'

In [3]:
def get_product_price_and_name():
    response = requests.get(url_to_scrape, headers=headers)

    soup = BeautifulSoup(response.content, 'html.parser')

    product_name = soup.find(id="productTitle").get_text().replace('\n', '')

    price_element = soup.find(id="priceblock_ourprice").get_text()
    price_string = str(price_element)[2:].replace(',', '')
    return float(price_string), product_name

In [12]:
def send_notification(product_name, last_price, curr_price):
    if last_price == curr_price: return

    status = 'dropped'
    if(last_price < curr_price): status = 'rose'

    bot_message = {
        'text' : 'Price of {} {} from Rs. {} to Rs. {}'
        .format(product_name, status, last_price, curr_price)}

    message_headers = {'Content-Type': 'application/json'}

    http_obj = Http()

    response = http_obj.request(
        uri=slack_url,
        method='POST',
        headers=message_headers,
        body=dumps(bot_message),
    )

In [10]:
def send_start_notification(product_name, initial_price):
    bot_message = {
        'text' : ('Started monitoring the price of {}.\nThe current price is '
         'Rs.{}, you will be notified if there is a price drop or rise '
         'on this item.')
        .format(product_name, initial_price)}

    message_headers = {'Content-Type': 'application/json'}

    http_obj = Http()

    response = http_obj.request(
        uri=slack_url,
        method='POST',
        headers=message_headers,
        body=dumps(bot_message),
    )

In [6]:
def send_error_notification():
    bot_message = {
        'text' : 'The script encountered an error.'
        .format(product_name, initial_price)}

    message_headers = {'Content-Type': 'application/json'}

    http_obj = Http()

    response = http_obj.request(
        uri=slack_url,
        method='POST',
        headers=message_headers,
        body=dumps(bot_message),
    )

In [13]:
try:
    initial_price, product_name = get_product_price_and_name()
    last_price = initial_price
    send_start_notification(product_name, initial_price)
    time.sleep(sleep_time)
    while(True):
        curr_price, product_name = get_product_price_and_name()
        send_notification(product_name, last_price, curr_price)
        time.sleep(sleep_time)
except:
    send_error_notification()